<a href="https://colab.research.google.com/github/patriciasbar/ztm-machine-learning/blob/main/dog_breed_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dog-breed Classification Project using TensorFlow

## 1. Problem
Identifying the breed of a dog given an image of a dog.

## 2. Data
Sourced from: https://www.kaggle.com/competitions/dog-breed-identification

## 3. Evaluation
"Submissions are evaluated on Multi Class Log Loss between the predicted probability and the observed target."

> _Multi-class log loss (or categorical log loss) is a way to measure how bad a classification model’s predictions are when there are three or more classes. It compares the predicted probabilities with the actual class and penalizes wrong guesses more when they are more confident. The lower the log loss, the better the model._

## 4. Features
* We are dealing with images (unstructured data) so it's probably best we use deep learning/transfer learning.
* There are 120 breeds of dogs (this means there are 120 different classes).
* Train Data: around 10.000+ images (these images have labels).
* Test Data: around 10.000+ images (no labels - taks is to predict them).

In [109]:
## unzip the uploaded into Google Drive - Colab Notebooks folder
#!unzip "drive/MyDrive/Colab Notebooks/dog-breed-identification.zip" -d "drive/MyDrive/Colab Notebooks/"


In [110]:
#pip install tf_keras

In [111]:
%env TF_USE_LEGACY_KERAS=1

env: TF_USE_LEGACY_KERAS=1


In [112]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
import os
from sklearn.model_selection import train_test_split

# print("TensorFlow version:", tf.__version__)
# print("TF Hub version:", hub.__version__)

## to read labels data
labels_csv = pd.read_csv("drive/MyDrive/Colab Notebooks/labels.csv")

In [113]:
#Check for GPU availability
print("GPU, available (Yes!)" if tf.config.list_physical_devices else "Not available!")

print("Is KerasLayer a Keras Layer:", isinstance(hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"), tf.keras.layers.Layer))

GPU, available (Yes!)
Is KerasLayer a Keras Layer: True


In [183]:
# labels_csv[:2]

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo


## Getting the data ready (turning into Tensors)

In [115]:
# labels_csv.describe()
# labels_csv.value_counts("breed").median()

## Getting filepaths and check if filepaths and amount of target data are matching

In [116]:
def is_data_matching(filepath, file_dir="drive/My Drive/Colab Notebooks/train/"):
  status = False;
  if file_dir:
    return len(os.listdir(file_dir)) == len(filepath)

def get_filepaths(labels_csv, file_dir="drive/My Drive/Colab Notebooks/train/"):
      return [file_dir + fname + ".jpg" for fname in labels_csv["id"]]


In [117]:
# data_status = is_data_matching(get_filepaths(labels_csv))
# print("Files check are ok?", data_status)

# Turning images into data

In [118]:
#transform labels into an np array
labels = labels_csv["breed"].to_numpy()

# labels_csv.head(5) #index4 = golden_retriever

In [119]:
# get unique breeds
unique_breeds = np.unique(labels)

# unique_breeds #index49 golden retriever


In [120]:
# # get an array of booleans
boolean_labels = [unique_breeds == label for label in labels]

# # golden retriever at label index 4 in the unique breed show as index 49
# boolean_labels[4], unique_breeds[49]


In [121]:
# convert bool arrays into integers
onehot_labels = [np.array(boolean_label).astype(int) for boolean_label in boolean_labels]


# Experimentation (~ 1000 images and increase as we go)

In [182]:
# #Set X & y variables
X = get_filepaths(labels_csv)
y = onehot_labels

# len(X),len(y)

['drive/My Drive/Colab Notebooks/train/000bec180eb18c7604dcecc8fe0dba07.jpg',
 'drive/My Drive/Colab Notebooks/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg',
 'drive/My Drive/Colab Notebooks/train/001cdf01b096e06d78e9e5112d419397.jpg',
 'drive/My Drive/Colab Notebooks/train/00214f311d5d2247d5dfe4fe24b2303d.jpg',
 'drive/My Drive/Colab Notebooks/train/0021f9ceb3235effd7fcde7f7538ed62.jpg',
 'drive/My Drive/Colab Notebooks/train/002211c81b498ef88e1b40b9abf84e1d.jpg',
 'drive/My Drive/Colab Notebooks/train/00290d3e1fdd27226ba27a8ce248ce85.jpg',
 'drive/My Drive/Colab Notebooks/train/002a283a315af96eaea0e28e7163b21b.jpg',
 'drive/My Drive/Colab Notebooks/train/003df8b8a8b05244b1d920bb6cf451f9.jpg',
 'drive/My Drive/Colab Notebooks/train/0042188c895a2f14ef64a918ed9c7b64.jpg',
 'drive/My Drive/Colab Notebooks/train/004396df1acd0f1247b740ca2b14616e.jpg',
 'drive/My Drive/Colab Notebooks/train/0067dc3eab0b3c3ef0439477624d85d6.jpg',
 'drive/My Drive/Colab Notebooks/train/00693b8bc2470375cc744a639

In [123]:
# Set number of images to user for experimenting
NUM_IMAGES = 1000 #@param {type:"slider", min:1000, max:10000}

In [124]:
# Split data into training and validations set
# X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size=0.2, random_state=42)

# len(X_train), len(X_val), len(y_train), len(y_val)

# Preprocessing Images (turning images into Tensors)

In [125]:
# # convert image to NumPy
# image = plt.imread(X_train[4])  #Reads an image from a file into an array.

# X_train[4], y_train[4]

# image.shape, image

In [126]:
#turn image into tensor
# tf.constant(image[4]), image[4]

In [127]:
# define image size (height / width)
IMG_SIZE = 224 #@param {type:"slider", min:224, max: 230}

In [128]:
## create a function for preprocessing images
def process_image(image_path, im_size=IMG_SIZE):
  """
  Takes an image file path and turns the image into a Tensor
  """
  # read an image file
  image = tf.io.read_file(image_path)
  # turn the jpeg image into numerical Tensor with RGB (3 colour channels)
  image = tf.image.decode_jpeg(image, channels=3)
  # convert the colour channel values (normalization) from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # resize the image as per constant `IMG_SIZE`
  image = tf.image.resize(image, size=[im_size, im_size])

  return image

In [129]:
## Turning data into batches

# create a function to return a tuple of Tensors (image, label)
def get_image_label(image_path=X, label=y):
  """
  Takes an image file path name and the associated label,
  processes the image and return a tuple (image, label).
  """
  return process_image(image_path), label



In [130]:
# (process_image(X[4]), tf.constant(y[4]))

#print(get_image_label(X[4], tf.constant(y[4]))) # returns a tuple

In [131]:
# define the batch size
BATCH_SIZE = 32 #@param {type:"slider", min:32, max:50}

In [132]:
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it is valid data.
  Also accepts test data as inputs (no labels(y)).
  """
  # If the data is a test dataset
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) # only filepaths
    data_batch = data.map(process_image).batch(batch_size)
    return data_batch

  # If the data is a valid dataset
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    data_batch = data.map(get_image_label).batch(batch_size)
    return data_batch

  # the data is a train dataset
  else:
    print("Creating training data batches...")
    # turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    # Shuffling pathnamesand labels before mapping image processor function
    data = data.shuffle(buffer_size=len(X))
    # Create (image label) tuples
    data = data.map(get_image_label)
    # turn the data into data batches
    data_batch = data.batch(batch_size)
    return data_batch



In [133]:
# # create training and validation data batches
# train_data = create_data_batches(X_train, y_train)

# val_data = create_data_batches(X_val, y_val, valid_data=True)

In [134]:
# train_data.element_spec, val_data.element_spec

In [135]:
def show_25_images(images, labels):
  """
  Displays a plot of 25 images and their labels from a data batch.
  """
  plt.figure(figsize=(10,10))
  #loop through the images
  for i in range(25):
    #create subplots (5 rows, 5 columns)
    ax = plt.subplot(5, 5, i+1)
    # display an image
    plt.imshow(images[i])
    # add the image label as the title
    plt.title(unique_breeds[labels[i].argmax()],fontsize=10)
    plt.axis("off")

In [136]:
# un-batch the data to visualize it
# train_images, train_labels = next(train_data.as_numpy_iterator())

In [137]:
# show_25_images(train_images, train_labels)

# Building a model

Things that need to be defined:
* The `INPUT_SHAPE` in the form of Tensors.
* The `OUTPUT_SHAPE` in the form of Tensors.
* The URL of the model to be used.

In [138]:
# setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3]

# setup output shape of the model
OUTPUT_SHAPE = len(unique_breeds)

# setup model URL from TensorFlor HUB
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5"

#MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"


# Keras Deep Learning Model

In [139]:
# # create a function which builds a Keras model
# def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
#   print("Building model with ", MODEL_URL)

#   # Setup the model layers
#   # model = tf.keras.Sequential([
#   #     tf.keras.layers.Lambda(lambda x: hub.KerasLayer(MODEL_URL)(x)), # Layer1 input layer
#   #     tf.keras.layers.Dense(units=OUTPUT_SHAPE,
#   #                          activation="softmax") # Layer2 output layer
#   # ])


#   # Compile the model
#   model.compile(
#       loss=tf.keras.losses.CategoricalCrossentropy(),
#       optimizer=tf.keras.optimizers.Adam(),
#       metrics=["Accuracy"]
#   )

#   # Build the model
#   model.build(INPUT_SHAPE)

#   return model

In [140]:
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", model_url)

  model = tf.keras.Sequential([
      hub.KerasLayer(model_url, input_shape=input_shape),
      tf.keras.layers.Dense(units=output_shape, activation="softmax")
  ])

  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["accuracy"]
  )

  return model


In [141]:
# Create a model and check its details
# model = create_model()

In [142]:
# model.summary()

# Create Callback functions

**Callbacks** are like helpers that watch training for you and do smart things while the model trains.

Common examples:

* Stop early if the model stops improving (`EarlyStopping`).
* Save checkpoints of your model while it trains (`ModelCheckpoint`).
* Log data for TensorBoard (`TensorBoard` callback).


In [143]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [144]:
import datetime

# create a function to build a TensorBoard callback:
def create_tensorboard_callback():
  # create a log dir for storing TensorBoard logs
  logdir = os.path.join("drive/MyDrive/Colab Notebooks/logs",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [145]:
# create an EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_Accuracy",
                                                  mode="max",
                                                  patience=3)

# Train and return a trained model

In [146]:
NUM_EPOCHS=100 #@param{type:"slider", min:10, max:100, step:10}

In [147]:
# build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version
  """
  # create a model
  model = create_model()

  # create new TensorBoard session everytime a model is trained
  tensorboard = create_tensorboard_callback()

  # fit the model to the data passing it the callbacks
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1,
            callbacks=[tensorboard, early_stopping])

  #return the fitted model
  return model

In [148]:
# model = train_model()

In [149]:
# %tensorboard --logdir /content/drive/MyDrive/Colab\ Notebooks/logs

# Making and evaluating predictions using a trained model

In [150]:
# make the predictions on the validation set
# predictions = model.predict(val_data,
#                            verbose=1)
# predictions

In [151]:
# Check one sample pred
# index = 4   #49 unique breeds
# #print(predictions[index])
# print(f"Max value (proba of predictions):  {np.max(predictions[index])}")
# print(f"Sum: {np.sum(predictions[index])}")
# print(f"Max index: {np.argmax(predictions[index])}")
# print(f"Predicted label: {unique_breeds[np.argmax(predictions[index])]}")


In [152]:
# turn proba into labels
def get_pred_label(pred_proba):
  """
  Turns an array of predicition probabilities into a label.
  """
  return unique_breeds[np.argmax(pred_proba)]

In [153]:
# pred_label = get_pred_label(predictions[4])
# pred_label

In [154]:
# unbatch data -  to compare preds to true labels
def unbatch_data(data):
  """
  Takes a batched dataset (image, label) Tensors
  and returns separate arrays of images and labels.
  """
  images_ = []
  labels_ = []

  for image, label in data.unbatch().as_numpy_iterator():
    images_.append(image)
    labels_.append(unique_breeds[np.argmax(label)])

  return images_, labels_

In [155]:
# val_images, val_labels = unbatch_data(val_data)
# val_images[4], get_pred_label(val_labels[4])

## Visualization: Plot Predictions and Truth labels

In [156]:
def plot_pred(predictions_proba, labels, images, n=4):
  """
  View the predictions, truth labels and image for sample n
  """
  pred_prob, true_label, image = predictions_proba[n], labels[n], images[n]

  # get the pred label
  pred_label = get_pred_label(pred_prob)

  # plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  #change the colour
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  # change plot title to be predicted, probability of prediction and truth label
  plt.title("{} {:2.0f}% {}".format(pred_label,
                                     np.max(pred_prob)*100,
                                            true_label),
            color = color)


In [157]:
# plot_pred(predictions_proba=predictions,
#           labels=val_labels,
#           images=val_images,
#           n=4)

# Create a function to plot top `top_n` predictions along with their truth label

In [158]:
def plot_topn_predictions(prediction_probabilities, labels, n=1, top_n=5):
  """
  Plots top `top_n` highest predictions along with their truth label.
  """

  pred_prob, true_label = predictions[n], labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)
  # Find the top `n` prediction confidence indexes
  topn_pred_prob = pred_prob.argsort()[-1*top_n:][::-1]
  # Find the top `n` prediction confidence values
  topn_pred_vals = pred_prob[topn_pred_prob]
  # Find the top `n` prediction labels
  topn_pred_labels = unique_breeds[topn_pred_prob]
  # Setup plot
  top_plot = plt.bar(np.arange(len(topn_pred_labels)),
                     topn_pred_vals,
                     color="lightgrey",
                     )
  plt.xticks(np.arange(len(topn_pred_labels)),
             labels = topn_pred_labels,
             rotation="vertical")
  # Change colour of true label
  print(true_label)
  print(topn_pred_labels)
  if np.isin(true_label, topn_pred_labels):
    top_plot[np.argmax(topn_pred_labels == true_label)].set_color("darkgreen")
  else:
    pass


In [159]:
# plot_topn_predictions(predictions, val_labels, n=9)


In [160]:
# i_multiplier = 0
# num_rows = 3
# num_cols = 2
# num_images = num_rows*num_cols
# plt.figure(figsize=(10*num_cols, 5*num_rows))
# for i in range(num_images):
#   plt.subplot(num_rows, 2*num_cols, 2*i+1)
#   plot_pred(predictions,
#             val_labels,
#             val_images,
#             n=i+i_multiplier)
#   plt.subplot(num_rows, 2*num_cols, 2*i+2)
#   plot_topn_predictions(prediction_probabilities=predictions,
#                         labels=val_labels,
#                         n=i+i_multiplier)
# plt.show()


# Saving and Reloading a trained model

In [161]:
# Create a function to save a model
def save_model(model, suffix=None):
  """
  Saves a pre trained model.
  """
  # Create a model dir pathname with current time
  modeldir = os.path.join("drive/MyDrive/Colab Notebooks/models",
                          datetime.datetime.now().strftime("%Y%m%d-%H-%M%s"))
  model_path = modeldir + "-" + suffix + ".keras" # save format of model
  print(f"Saving model to: {modeldir}....")
  model.save(model_path)
  return model_path

In [162]:
# Create a functionl to load a trained model
def load_model(model_path, ):
  """
  Loads a trained saved model from a specified path.
  """
  import tensorflow_hub as hub
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={'KerasLayer': hub.KerasLayer},
                                     safe_mode=False)
  return model


In [163]:
 # save_model(model, suffix="1000-images-mobilenetv2-Adam")

In [164]:
# loaded_model = load_model("drive/MyDrive/Colab Notebooks/models/20250418-11-571744977448-1000-images-mobilenetv2-Adam.keras")

In [165]:
# model.evaluate(val_data)
# loaded_model.evaluate(val_data)


# Training Model in Full Dataset (10k)

In [166]:
len(X), len(y)

(10222, 10222)

In [167]:
# Turn full training data in a data batch
full_data = create_data_batches(X, y)

Creating training data batches...


In [168]:
# Instantiate a new model for training on the full dataset
full_model = create_model()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5


In [169]:
 #Create full model callbacks

# TensorBoard callback
full_model_tensorboard = create_tensorboard_callback()

# Early stopping callback
# Note: No validation set when training on all the data, therefore can't monitor validation accruacy
full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                             patience=3)

In [170]:
# %tensorboard --logdir drive/MyDrive/Colab\ Notebooks/logs

In [171]:
# Fit the full model to the full training data
# full_model.fit(x=full_data,
#                epochs=NUM_EPOCHS,
#                callbacks=[full_model_tensorboard,
#                           full_model_early_stopping])

In [172]:
# Save model to file
# save_model(full_model, suffix="all-images-Adam")

In [173]:
loaded_full_model = load_model("drive/MyDrive/Colab Notebooks/models/20250418-12-301744979450-all-images-Adam.keras")

Loading saved model from: drive/MyDrive/Colab Notebooks/models/20250418-12-301744979450-all-images-Adam.keras


In [174]:
loaded_full_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_6 (KerasLayer)  (None, 1664)              3766048   
                                                                 
 dense_3 (Dense)             (None, 120)               199800    
                                                                 
Total params: 3965848 (15.13 MB)
Trainable params: 199800 (780.47 KB)
Non-trainable params: 3766048 (14.37 MB)
_________________________________________________________________


# Making predictions on the Test dataset

In [187]:
# read test data (X_test)
## Set X_test and y_test variables
test_path = "drive/My Drive/Colab Notebooks/test/"

# appends the test_path to all filenames
test_filename = [test_path + fname for fname in os.listdir(test_path)]

In [192]:
# create test data batches
test_data = create_data_batches(X=test_filename, batch_size=BATCH_SIZE, test_data=True)

test_data

Creating test data batches...


<_BatchDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>

In [193]:
#test_pred = loaded_full_model.predict(test_data, verbose=1)

324/324 [==============================] - 1911s 6s/step


In [194]:
#np.savetxt("drive/MyDrive/Colab Notebooks/preds_array.csv", test_pred, delimiter=",")

In [196]:
test_preds = np.loadtxt("drive/MyDrive/Colab Notebooks/preds_array.csv", delimiter=",")

In [197]:
test_preds

array([[1.35565770e-07, 2.56752342e-10, 3.54651593e-06, ...,
        4.76179096e-09, 5.62744518e-10, 1.56284159e-08],
       [7.97400298e-06, 8.66927257e-06, 7.65938410e-07, ...,
        8.37348011e-07, 2.46277998e-09, 4.53306939e-07],
       [8.77984276e-05, 1.11986864e-07, 6.12394544e-08, ...,
        1.34271630e-07, 1.43495004e-06, 7.91812718e-01],
       ...,
       [4.18739692e-05, 4.96224629e-06, 1.25262829e-06, ...,
        9.72100042e-06, 2.11841871e-06, 6.36298792e-05],
       [5.66320298e-12, 1.44545055e-07, 3.13483390e-08, ...,
        3.58491408e-04, 1.90916776e-08, 3.73760400e-09],
       [1.20367231e-05, 7.82432267e-04, 1.05176616e-06, ...,
        1.32429728e-03, 5.08402742e-08, 2.96660858e-07]])

In [200]:
# Create predictions file to submit to Kaggle
# create pandas DataFrame with id col and unique breeds
preds_df = pd.DataFrame(columns=["id"] + list(unique_breeds))
preds_df[0]

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier


In [205]:
# add data to DataFrame
test_ids = [os.path.splitext(path)[0] for path in os.listdir(test_path)]
preds_df["id"] = test_ids
preds_df[:1]

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,e7ce78e874945f182a4f5149aa505b09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
# add the prediction probabilities
preds_df[list(unique_breeds)] = test_preds
preds_df[:2]

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,e7ce78e874945f182a4f5149aa505b09,1.355658e-07,2.567523e-10,3.546516e-06,1.078225e-12,2.833878e-08,1.221738e-07,6.855399e-08,1.455044e-07,1.099912e-10,...,9.949073e-07,8.873284e-09,2.550151e-08,1.069272e-11,1.630661e-07,1.463453e-10,1.256071e-08,4.761791e-09,5.627445e-10,1.562842e-08
1,e7be7b911a4cba9fdfa4105ec4776370,7.974003e-06,8.669273e-06,7.659384e-07,1.469063e-08,4.167904e-10,2.382131e-10,4.044440e-08,1.044386e-08,3.071878e-10,...,4.223003e-08,3.110691e-11,1.694002e-09,1.189198e-11,3.858442e-09,2.750516e-10,3.473071e-08,8.373480e-07,2.462780e-09,4.533069e-07


In [214]:
# save to csv
preds_df.to_csv("drive/MyDrive/Colab Notebooks/predictions_dog_breed_mobilenet_v2_130_224_20250420.csv", index=False)